# Greenblatt Magic formula

## The Magic Formula: Explanation and Formula

The **Magic Formula** is an investment strategy developed by **Joel Greenblatt** to identify high-quality companies that are also undervalued. It ranks companies based on two key financial ratios:

---

## 📌 Core Idea

> **Buy good companies at cheap prices.**

The strategy identifies:
- **"Good companies"** → those with high returns on capital (efficient use of capital).
- **"Cheap companies"** → those with high earnings yield (undervalued based on operating profits).

---

## 🧮 The original Formula

1. **Earnings Yield (EY):**
$$
\text{Earnings Yield} = \frac{\text{EBIT}}{\text{Enterprise Value}}
$$
- Measures how cheap the stock is.
- EBIT = Earnings Before Interest and Taxes
- Enterprise Value = Market Cap + Debt - Cash

2. **Return on Capital (ROC):**
$$
\text{Return on Capital} = \frac{\text{EBIT}}{\text{Net Working Capital} + \text{Net Fixed Assets}}
$$
- Measures the quality of the business (how efficiently it uses its capital).
### My filters
1. Add minimum revenue
2. Skip if net income is non-positive
3. Skip (net_income > 0 and cashflow < 0) or (net_income < 0 and cashflow > 0)
4. Operating expenses must be non-negative
5. Exclude extreme accounting adjustments (per filteration in the code)
6. Remove absurdly high EBIT margins (e.g., > 200%)
---

## 🔍 Implementation Steps

1. **Filter the universe**: Remove financials, utilities, and companies with very small market cap.
2. **Rank all remaining stocks** by:
   - Earnings Yield (high = better)
   - Return on Capital (high = better)
3. **Compute combined rank**:
   $$
   \text{Combined Rank} = \text{Rank}_{EY} + \text{Rank}_{ROC}
   $$
4. **Sort by Combined Rank** (lowest = best overall).
5. **Pick top N stocks** (e.g., top 20–30).
6. **Hold for 1 year**, rebalance annually.

---

## ✅ Why It Works

- Avoids paying too much for popular stocks.
- Focuses on operationally efficient, consistently profitable companies.
- Enforces a disciplined, rules-based approach.

---

## ⚠️ Notes and Caveats

- Avoids subjective judgement; however, **screening accuracy** depends on **quality of financial data**.
- May underperform in short-term or irrational markets.
- Works best over a multi-year horizon (3–5+ years).
- I have separated this into US and International stocks.
---


In [1]:
import os
from tenacity import retry, stop_after_attempt, wait_exponential
import requests
import pandas as pd
from tqdm import tqdm

api_key = os.getenv('financial_modeling_prep_api_key')
assert api_key is not None

In [2]:
# Retry settings: 5 attempts with exponential backoff
@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=1, min=10, max=60))
def fetch_json(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

def get_magic_formula(api_key, symbols, minimum_margin=0.4, minimum_marketcap=500_000_000, minimum_revenue=100_000_000):
    import pandas as pd
    from tqdm import tqdm

    base_url = "https://financialmodelingprep.com/stable"
    results = []

    for symbol in tqdm(symbols, desc="Processing symbols"):
        try:
            # Step 1: Check market cap
            ev_url = f"{base_url}/enterprise-values/?symbol={symbol}&apikey={api_key}"
            ev_data = fetch_json(ev_url)[0]
            market_cap = ev_data.get('marketCapitalization')  
            
            if not ev_data or market_cap < minimum_marketcap:
                continue

            # Step 2: Exclude financials and real estate sectors
            profile_url = f"{base_url}/profile?symbol={symbol}&apikey={api_key}"
            profile_data = fetch_json(profile_url)
            sector = profile_data[0].get('sector') 
            company_name = profile_data[0].get('companyName')
            
            if not profile_data or sector in ['Financial Services', 'Real Estate']:
                continue

            # Step 3: Fetch income statement
            income_url = f"{base_url}/income-statement/?symbol={symbol}&apikey={api_key}"
            income_data = fetch_json(income_url)[0]
            net_income = income_data.get('netIncome')
            revenue = income_data.get('revenue')

            # Filter: minimum revenue
            if revenue < minimum_revenue:
                continue

            # Skip if net income is non-positive (net_income > 0 and cashflow < 0) or (net_income < 0 and cashflow > 0)
            if net_income is None or net_income <= 0:
                continue

            cash_url = f"{base_url}/cash-flow-statement/?symbol={symbol}&apikey={api_key}"
            cash_data = fetch_json(cash_url)[0]
            cashflow = cash_data['operatingCashFlow']
            
            #if abs(net_income - cashflow) > 0.5 * net_income:
            if (net_income > 0 and cashflow < 0) or (net_income < 0 and cashflow > 0):
                print(f"Cashflow mismatch for {symbol}")
                continue  # net income not supported by cash flow

            # Step 4: Fetch balance sheet
            balance_url = f"{base_url}/balance-sheet-statement/?symbol={symbol}&apikey={api_key}"
            balance_data = fetch_json(balance_url)[0]

            # Step 5: Fetch dividend
            dividend = f"{base_url}/dividends/?symbol={symbol}&apikey={api_key}"
            dividend_data = fetch_json(dividend)
            dividend = 0
            if dividend_data:
                dividend_data = dividend_data[0]
                dividend = dividend_data.get('yield')
                div_date = dividend_data.get('date')

            # Extract relevant fields
            ebit = income_data.get('ebit')
            
            total_assets = balance_data.get('totalAssets')
            current_liabilities = balance_data.get('totalCurrentLiabilities')
            enterprise_value = ev_data.get('enterpriseValue')
            operating_expenses = income_data.get('operatingExpenses')
            other_expenses = income_data.get('otherExpenses')

            # Ensure all required fields are present
            if None in (ebit, revenue, total_assets, current_liabilities, enterprise_value):
                print(f"missing data for {symbol}")
                continue

            # Filter: Operating expenses must be non-negative
            if operating_expenses is not None and operating_expenses < 0:
                continue

            # Filter: exclude extreme accounting adjustments
            if other_expenses is not None and other_expenses < -revenue:
                continue

            # Filter: remove absurdly high EBIT margins (e.g., > 200%)
            ebit_margin = ebit / revenue
            if ebit_margin < minimum_margin or ebit_margin > 2:
                continue

            # Compute Greenblatt metrics
            earnings_yield = ebit / enterprise_value
            capital = total_assets - current_liabilities
            if capital <= 0:
                continue
            return_on_capital = ebit / capital

            # Append result
            results.append({
                'symbol': symbol,
                'company_name': company_name,
                'dividend': dividend,
                'div date': div_date,
                'sector' : sector,
                'ebit_margin': ebit_margin,
                'earnings_yield': earnings_yield,
                'return_on_capital': return_on_capital
            })

        except Exception as e:
            print(f'Symbol: {symbol}\n')
            print(e)

    # Step 6: Compile DataFrame and rank
    df = pd.DataFrame(results)
    if df.empty:
        return df

    df['ey_rank'] = df['earnings_yield'].rank(ascending=False)
    df['roc_rank'] = df['return_on_capital'].rank(ascending=False)
    df['combined_rank'] = df['ey_rank'] + df['roc_rank']
    df_sorted = df.sort_values(by='combined_rank')

    return df_sorted #[['symbol', 'ebit_margin', 'earnings_yield', 'return_on_capital', 'combined_rank']]


In [3]:
symbols = pd.read_csv("russel1000.csv")
symbols = symbols['Ticker'].to_list()

top_stocks = get_magic_formula(api_key, symbols, minimum_marketcap=50_000_000, minimum_margin=0.05, minimum_revenue=0)

print(top_stocks)
russel1000_filtered = top_stocks
russel1000_filtered.head(30).to_csv('russel1000_magic_formula.csv')

Processing symbols:   1%|▋                                                            | 11/1012 [00:05<06:31,  2.56it/s]

Symbol: BRKB

'<' not supported between instances of 'NoneType' and 'int'


Processing symbols:  10%|██████                                                      | 102/1012 [01:31<15:27,  1.02s/it]

Cashflow mismatch for CEG


Processing symbols:  13%|████████                                                    | 135/1012 [01:47<04:41,  3.12it/s]

...

Processing symbols:  99%|███████████████████████████████████████████████████████████▏| 999/1012 [15:36<00:05,  2.39it/s]

Symbol: CWENA

list index out of range


Processing symbols: 100%|███████████████████████████████████████████████████████████| 1012/1012 [15:40<00:00,  1.08it/s]

Symbol: --

list index out of range
Symbol: FAH5

list index out of range
Symbol: ESH5

list index out of range


    symbol                  company_name  dividend    div date  \
77      MO            Altria Group, Inc.  6.879110  2025-06-16   
274    JBL                    Jabil Inc.  0.192158  2025-05-15   
437   BBWI       Bath & Body Works, Inc.  2.958580  2025-06-06   
460    HRB               H&R Block, Inc.  2.574665  2025-06-04   
248    PHM              PulteGroup, Inc.  0.844213  2025-06-17   
..     ...                           ...       ...         ...   
446    TFX         Teleflex Incorporated  1.072048  2025-05-20   
60    GILD         Gilead Sciences, Inc.  2.861860  2025-06-13   
495   FRPT                Freshpet, Inc.  0.000000  2025-06-05   
551     ZI    ZoomInfo Technologies Inc.  0.000000  2025-05-15   
38     AMD  Advanced Micro Devices, Inc.  0.027778  1995-04-28   

                 sector  ebit_margin  earnings_yield  return_on_capital  \
77   Consumer Defensive     0.723048        0.132416           0.560009   
274          Technology     0.066614        0.133316     

In [111]:
russel1000_filtered.head(30)

,symbol,company_name,dividend,sector,ebit_margin,earnings_yield,return_on_capital,ey_rank,roc_rank,combined_rank


In [ ]:
symbols = pd.read_csv("all_listed_companies.csv")
symbols = symbols['Symbol'].to_list()

top_stocks = get_magic_formula(api_key, symbols, minimum_margin=0.05)
print(top_stocks)
world_filtered = top_stocks
world_filtered[~world_filtered['symbol'].isin(russel1000_filtered['symbol'].to_list())].head(30).to_csv('world_magic_formula.csv')

## Tests

In [ ]:
base_url = "https://financialmodelingprep.com/stable"
profile_url = f"{base_url}/price-target-consensus?symbol=NVDA&apikey={api_key}"
response = requests.get(profile_url)
response.raise_for_status()
profile_data = response.json()

In [ ]:
profile_data[0]#['yield']

In [ ]:
profile_data[0] #left tilted median bigger than average, 